In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

### Some intuition

In [141]:
# Randomly generated customer IDs and purchased months
df = pd.DataFrame([
    {'customer': customer_id, 'month': month}
    for month, customer_id in
    zip(np.random.randint(1, 13, 100), np.random.randint(1, 31, 100))
])

In [142]:
cohort = df.sort_values(by='month').groupby('customer')['month'].first()
cohort = cohort.to_frame(name='cohort')

In [143]:
cohort.head()

,cohort
customer,
1,6
2,7
3,12
4,7
5,8


In [144]:
df_cohort = pd.merge(df, cohort, right_index=True, left_on='customer')
df_cohort.head()

,customer,month,cohort
0,15,12,3
30,15,5,3
46,15,3,3
48,15,11,3
1,22,5,4


In [145]:
df_cohort.groupby('cohort').size()

cohort
1      7
2     36
3     11
4     16
5      2
6      9
7     14
8      4
12     1
dtype: int64

In [202]:
def cohort_generator(df, group_by, customer_column, period_column):
    results = []
    
    for cohort, data in df.groupby(group_by):
        cohort_size = data[customer_column].nunique()
        unique_periods = sorted(df[period_column].unique())

        for period in unique_periods:
            unique = data[data[period_column] >= period][customer_column].nunique()
            if period >= cohort and unique >= 0:
                retention = unique / cohort_size
                results.append({'cohort': cohort,
                               'period': period,
                               'retention': retention})
            
    c = pd.DataFrame.from_records(results)
    return c.pivot(index='period', columns='cohort')

In [163]:
cohort_generator(df_cohort, 'cohort', 'customer', 'month')

retention                                                    
cohort        1      2         3    4    5         6    7    8    12
period                                                              
1            1.0    NaN       NaN  NaN  NaN       NaN  NaN  NaN  NaN
2            1.0  1.000       NaN  NaN  NaN       NaN  NaN  NaN  NaN
3            1.0  1.000  1.000000  NaN  NaN       NaN  NaN  NaN  NaN
4            1.0  1.000  1.000000  1.0  NaN       NaN  NaN  NaN  NaN
5            1.0  0.875  1.000000  1.0  1.0       NaN  NaN  NaN  NaN
6            1.0  0.875  1.000000  1.0  1.0  1.000000  NaN  NaN  NaN
7            1.0  0.875  1.000000  1.0  1.0  0.666667  1.0  NaN  NaN
8            1.0  0.750  1.000000  1.0  0.0  0.666667  0.8  1.0  NaN
9            1.0  0.750  1.000000  0.6  0.0  0.666667  0.8  0.5  NaN
10           0.0  0.750  1.000000  0.4  0.0  0.666667  0.6  0.5  NaN
11           0.0  0.750  0.333333  0.2  0.0  0.333333  0.2  0.5  NaN
12           0.0  0.625  0.333333  0.2  0.0  0.000000  0.2  0.0  1.0

### Actual data

In [7]:
orders = pd.read_csv('orders.csv')[['customer_id', 'created_at']]

In [8]:
orders['created_at'] = pd.to_datetime(orders['created_at'])

In [226]:
orders.head()

,customer_id,created_at
0,1,2013-04-07 12:00:00
1,2,2013-04-04 12:00:00
2,4,2013-04-06 12:00:00
3,3,2013-04-08 12:00:00
4,10,2013-04-04 12:00:00


In [9]:
%%time
# Extract year and date only
orders['created_at_month_year'] = orders['created_at'].map(lambda x: x.strftime('%Y-%m'))

CPU times: user 27.1 s, sys: 441 ms, total: 27.6 s
Wall time: 27.6 s


In [228]:
orders.head()

,customer_id,created_at,created_at_month_year
0,1,2013-04-07 12:00:00,2013-04
1,2,2013-04-04 12:00:00,2013-04
2,4,2013-04-06 12:00:00,2013-04
3,3,2013-04-08 12:00:00,2013-04
4,10,2013-04-04 12:00:00,2013-04


In [10]:
%%time
# Assign a cohort to each 
cohort = orders.sort_values(by='created_at_month_year').groupby('customer_id')['created_at_month_year'].first()
cohort = cohort.to_frame(name='cohort')

CPU times: user 4.43 s, sys: 213 ms, total: 4.65 s
Wall time: 4.66 s


In [230]:
cohort.head()

,cohort
customer_id,
1,2013-04
2,2013-04
3,2013-04
4,2013-04
5,2013-04


In [11]:
%%time
df_cohort = pd.merge(orders[['customer_id',
                             'created_at_month_year']],
                     cohort, right_index=True, left_on='customer_id')
assert len(orders) == len(df_cohort)

CPU times: user 1.44 s, sys: 165 ms, total: 1.61 s
Wall time: 1.61 s


In [232]:
df_cohort.head()

,customer_id,created_at_month_year,cohort
0,1,2013-04,2013-04
287846,1,2013-10,2013-04
1646408,1,2015-03,2013-04
1871621,1,2015-04,2013-04
1,2,2013-04,2013-04


In [239]:
# Number of unique customers in each cohort
df_cohort.groupby('cohort').agg({'customer_id': pd.Series.nunique}).head()

,customer_id
cohort,
2013-04,28874
2013-05,30851
2013-06,29953
2013-07,31013
2013-08,30851


In [235]:
%%time
cohort = cohort_generator(df_cohort, 'cohort', 'customer_id', 'created_at_month_year')

CPU times: user 46.3 s, sys: 960 ms, total: 47.2 s
Wall time: 47.3 s


In [236]:
cohort

retention                                                              \
cohort    2013-04   2013-05   2013-06   2013-07   2013-08   2013-09   2013-10   
period                                                                          
2013-04  1.000000       NaN       NaN       NaN       NaN       NaN       NaN   
2013-05  0.986839  1.000000       NaN       NaN       NaN       NaN       NaN   
2013-06  0.985212  0.984603  1.000000       NaN       NaN       NaN       NaN   
2013-07  0.983480  0.982821  0.984976  1.000000       NaN       NaN       NaN   
2013-08  0.981852  0.980390  0.983107  0.982781  1.000000       NaN       NaN   
2013-09  0.979497  0.977829  0.980937  0.980943  0.979450  1.000000       NaN   
2013-10  0.977627  0.975236  0.978299  0.978041  0.976532  0.977918  1.000000   
2013-11  0.974510  0.972221  0.975628  0.975333  0.973777  0.975208  0.973073   
2013-12  0.971220  0.968785  0.972557  0.972302  0.970017  0.972231  0.969655   
2014-01  0.967341  0.964896  0.968718  0.967949  0.966711  0.968617  0.965334   
2014-02  0.962769  0.960844  0.964144  0.963435  0.961849  0.964067  0.961045   
2014-03  0.957886  0.955723  0.959370  0.958727  0.957376  0.959684  0.956530   
2014-04  0.952137  0.949596  0.953627  0.952858  0.951314  0.954866  0.950919   
2014-05  0.945903  0.943503  0.947151  0.946603  0.944605  0.948175  0.944212   
2014-06  0.939046  0.936761  0.939205  0.939832  0.937150  0.940781  0.936988   
2014-07  0.931184  0.929954  0.931459  0.931835  0.929727  0.933655  0.930184   
2014-08  0.921625  0.920586  0.922645  0.923322  0.921721  0.923751  0.920832   
2014-09  0.910646  0.911283  0.913197  0.913714  0.911316  0.914082  0.909965   
2014-10  0.899598  0.899809  0.901679  0.902686  0.901462  0.902305  0.898388   
2014-11  0.886542  0.886551  0.888993  0.889466  0.888723  0.889257  0.884973   
2014-12  0.871788  0.873391  0.874003  0.874085  0.873489  0.874368  0.870622   
2015-01  0.854748  0.857606  0.855874  0.856544  0.856666  0.857372  0.853370   
2015-02  0.835215  0.839033  0.836544  0.837584  0.837023  0.838201  0.834892   
2015-03  0.815717  0.820135  0.818048  0.818721  0.815727  0.819365  0.814995   
2015-04  0.789915  0.794431  0.793744  0.790894  0.789893  0.793971  0.788617   
2015-05  0.761931  0.767593  0.766735  0.763905  0.762925  0.768443  0.762722   
2015-06  0.731662  0.736216  0.736120  0.733434  0.731743  0.737696  0.733634   
2015-07  0.697583  0.701890  0.702133  0.700932  0.698292  0.704975  0.700322   
2015-08  0.656785  0.664452  0.661570  0.659369  0.658131  0.665763  0.661045   
2015-09  0.609891  0.620563  0.615297  0.614903  0.614956  0.619827  0.617285   
2015-10  0.562721  0.574309  0.565686  0.567439  0.565363  0.568169  0.569526   
2015-11  0.505749  0.517033  0.512102  0.511398  0.509384  0.510857  0.513157   
2015-12  0.443860  0.456744  0.448536  0.448715  0.447960  0.448526  0.451048   
2016-01  0.371580  0.382289  0.377291  0.374424  0.376681  0.376292  0.377556   
2016-02  0.288460  0.299374  0.296431  0.291878  0.292146  0.294757  0.297581   
2016-03  0.203817  0.211144  0.209061  0.203753  0.207740  0.207300  0.211448   
2016-04  0.099155  0.102622  0.104364  0.100055  0.101780  0.103014  0.104063   

                                        ...                                  \
cohort    2013-11   2013-12   2014-01   ...     2015-07   2015-08   2015-09   
period                                  ...                                   
2013-04       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-05       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-06       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-07       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-08       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-09       NaN       NaN       NaN   ...         NaN       NaN       NaN   
2013-10       NaN       NaN       NaN   ...         NaN 

### Revenue

In [16]:
orders = pd.read_csv('orders.csv')[['id', 'customer_id', 'created_at']]
orders.head()

,id,customer_id,created_at
0,1,1,2013-04-07 12:00:00
1,2,2,2013-04-04 12:00:00
2,3,4,2013-04-06 12:00:00
3,4,3,2013-04-08 12:00:00
4,5,10,2013-04-04 12:00:00


In [19]:
items = pd.read_csv('order_items.csv')[['order_id', 'product_id', 'quantity']]
items.head()

,order_id,product_id,quantity
0,1,1,1
1,2,6,1
2,3,2,1
3,4,1,1
4,5,1,1


In [36]:
# No order has more than one item
items.order_id.value_counts().head()

2047      1
650647    1
531881    1
529832    1
552359    1
Name: order_id, dtype: int64

In [22]:
products = pd.read_csv('products.csv')[['id', 'price']]
products.head()

,id,price
0,1,8999
1,2,4999
2,3,12999
3,4,9999
4,5,7999


In [29]:
items_products = pd.merge(items,
                          products,
                          how='left',
                          left_on='product_id',
                          right_on='id')[['order_id', 'quantity', 'price']]
items_products['total'] = items_products.quantity * items_products.price
items_products = items_products[['order_id', 'total']]
items_products.head()

,order_id,total
0,1,8999
1,2,7999
2,3,4999
3,4,8999
4,5,8999


In [40]:
combined = pd.merge(orders, items_products, how='left', left_on='id', right_on='order_id')
assert len(orders) == len(combined)
combined = combined[['customer_id', 'created_at', 'total']]
combined.head()

,customer_id,created_at,total
0,1,2013-04-07 12:00:00,8999.0
1,2,2013-04-04 12:00:00,7999.0
2,4,2013-04-06 12:00:00,4999.0
3,3,2013-04-08 12:00:00,8999.0
4,10,2013-04-04 12:00:00,8999.0


In [42]:
%%time
combined['created_at'] = pd.to_datetime(combined['created_at'])

CPU times: user 794 ms, sys: 57.3 ms, total: 851 ms
Wall time: 847 ms


In [43]:
%%time
# Extract year and date only
combined['created_at_month_year'] = combined['created_at'].map(lambda x: x.strftime('%Y-%m'))

CPU times: user 28.1 s, sys: 513 ms, total: 28.7 s
Wall time: 28.7 s


In [44]:
%%time
# Assign a cohort to each 
cohort = combined.sort_values(by='created_at_month_year').groupby('customer_id')['created_at_month_year'].first()
cohort = cohort.to_frame(name='cohort')

CPU times: user 4.39 s, sys: 278 ms, total: 4.67 s
Wall time: 4.66 s


In [45]:
cohort.head()

,cohort
customer_id,
1,2013-04
2,2013-04
3,2013-04
4,2013-04
5,2013-04


In [49]:
%%time
df_cohort = pd.merge(combined[['customer_id',
                             'created_at_month_year', 'total']],
                     cohort, right_index=True, left_on='customer_id')
assert len(orders) == len(df_cohort)

CPU times: user 1.49 s, sys: 187 ms, total: 1.68 s
Wall time: 1.68 s


In [50]:
df_cohort.head()

,customer_id,created_at_month_year,total,cohort
0,1,2013-04,8999.0,2013-04
287846,1,2013-10,9999.0,2013-04
1646408,1,2015-03,NaN,2013-04
1871621,1,2015-04,NaN,2013-04
1,2,2013-04,7999.0,2013-04


In [52]:
revenue = df_cohort.groupby(['cohort', 'created_at_month_year']).agg({'total': pd.np.sum})
revenue.head()

total
cohort  created_at_month_year             
2013-04 2013-04                216899526.0
        2013-05                 25514894.0
        2013-06                 26227225.0
        2013-07                 26419512.0
        2013-08                 27238296.0

In [58]:
revenue.reset_index().rename(columns={
    'created_at_month_year': 'period',
    'total': 'revenue'
}).pivot(index='cohort', columns='period')

revenue                                                      \
period       2013-04      2013-05      2013-06      2013-07      2013-08   
cohort                                                                     
2013-04  216899526.0   25514894.0   26227225.0   26419512.0   27238296.0   
2013-05          NaN  232107922.0   26940399.0   28880927.0   28575980.0   
2013-06          NaN          NaN  226205043.0   27377372.0   27899470.0   
2013-07          NaN          NaN          NaN  234239087.0   27902357.0   
2013-08          NaN          NaN          NaN          NaN  232414240.0   
2013-09          NaN          NaN          NaN          NaN          NaN   
2013-10          NaN          NaN          NaN          NaN          NaN   
2013-11          NaN          NaN          NaN          NaN          NaN   
2013-12          NaN          NaN          NaN          NaN          NaN   
2014-01          NaN          NaN          NaN          NaN          NaN   
2014-02          NaN          NaN          NaN          NaN          NaN   
2014-03          NaN          NaN          NaN          NaN          NaN   
2014-04          NaN          NaN          NaN          NaN          NaN   
2014-05          NaN          NaN          NaN          NaN          NaN   
2014-06          NaN          NaN          NaN          NaN          NaN   
2014-07          NaN          NaN          NaN          NaN          NaN   
2014-08          NaN          NaN          NaN          NaN          NaN   
2014-09          NaN          NaN          NaN          NaN          NaN   
2014-10          NaN          NaN          NaN          NaN          NaN   
2014-11          NaN          NaN          NaN          NaN          NaN   
2014-12          NaN          NaN          NaN          NaN          NaN   
2015-01          NaN          NaN          NaN          NaN          NaN   
2015-02          NaN          NaN          NaN          NaN          NaN   
2015-03          NaN          NaN          NaN          NaN          NaN   
2015-04          NaN          NaN          NaN          NaN          NaN   
2015-05          NaN          NaN          NaN          NaN          NaN   
2015-06          NaN          NaN          NaN          NaN          NaN   
2015-07          NaN          NaN          NaN          NaN          NaN   
2015-08          NaN          NaN          NaN          NaN          NaN   
2015-09          NaN          NaN          NaN          NaN          NaN   
2015-10          NaN          NaN          NaN          NaN          NaN   
2015-11          NaN          NaN          NaN          NaN          NaN   
2015-12          NaN          NaN          NaN          NaN          NaN   
2016-01          NaN          NaN          NaN          NaN          NaN   
2016-02          NaN          NaN          NaN          NaN          NaN   
2016-03          NaN          NaN          NaN          NaN          NaN   
2016-04          NaN          NaN          NaN          NaN          NaN   

                                                                          \
period       2013-09      2013-10      2013-11      2013-12      2014-01   
cohort                                                                     
2013-04   26811655.0   26953020.0   26306898.0   27117914.0   28049071.0   
2013-05   28521738.0   28876224.0   27650180.0   29597311.0   28330289.0   
2013-06   26903092.0   27643548.0   26991132.0   28005795.0   28217347.0   
2013-07   27995429.0   29258999.0   28589001.0   28323543.0   29144705.0   
2013-08   26891138.0   28408414.0   28407545.0   29164234.0   28222726.0   
2013-09  225802212.0   26460161.0   26729968.0   27601337.0   26987381.0   
2013-10          NaN  234676292.0   26828754.0   28158126.0   29055815.0   
2013-11          NaN          NaN  225629435.0   26484060.0   28729198.0   
2013-12          NaN          NaN          NaN  233756503.0   27495206.0   
2014-01          NaN          NaN          NaN          NaN 